# 🦌 Node Label Placement 🏷️

In [ ]:
import asyncio
import json
import pathlib

import ipywidgets as W
import traitlets as T

import ipyelk
from ipyelk.diagram import layout_options

In [ ]:
nlp = layout_options.NodeLabelPlacement()
nlp

ELKJSON build acceptable value for the label placement

In [ ]:
nlp.value

### Example connecting NodeLabelPlacement widget value to the label layoutOptions

In [ ]:
simple = ipyelk.ElkDiagram()
sizer = ipyelk.diagram.ElkTextSizer()


async def update_label_layout_options(node, options):
    "Recurse through the nodes and update layout options"
    for label in node.get("labels", []):
        if "layoutOptions" in label:
            label["layoutOptions"].update(options)
        else:
            label["layoutOptions"] = options

        # removing rigid x and y placement
        label.pop("x", None)
        label.pop("y", None)

        # updating label size with measured values from the DOM
        size = await sizer.measure(label.get("text"))
        label["height"] = size.height
        label["width"] = size.width
    for child in node.get("children", []):
        await update_label_layout_options(child, options)


async def update_json():
    elk_json = json.loads(pathlib.Path("simple.json").read_text(encoding="utf-8"))
    elk_json["properties"] = {"algorithm": "layered"}
    layoutOptions = {"nodeLabels.placement": nlp.value}
    await update_label_layout_options(elk_json, layoutOptions)
    simple.value = elk_json


def refresh(change=None):
    """Wrapper function for scheduling the async update_json function"""
    asyncio.create_task(update_json())


nlp.observe(refresh, "value")
refresh()
W.HBox([simple, nlp], layout={"height": "400px"})

## 🦌 Learn More 📖

- [🦌 Introducing ELK 👋](./00_Introduction.ipynb)
- [🦌 Linking ELK Diagrams 🔗](./01_Linking.ipynb)
- [🦌 ELK Transformer 🤖](./02_Transformer.ipynb)
- [🦌 ELK App 🚀](./03_App.ipynb)
- [🦌 Interactive ELK App 🕹️](./04_Interactive.ipynb)
- [🦌 Node Label Placement 🏷️](./100_node_label_placement.ipynb)
- [🦌 Text Sizer 📏](./101_text_sizer.ipynb)